In [ ]:
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import pandas as pd
import numpy as np
import random
import os, os.path, pickle, sys
import pyreadstat
from datetime import datetime

# Filter from October of last year to the harvest date

In [2]:
data_base = "/Users/ehsannorouzi/Documents/01_research_data/Ehsan/wheat/Data/"
raw_dir = os.path.join(data_base, "00_raw_by_location/")

In [10]:
df_all = pd.read_csv(os.path.join(raw_dir, "raw_all_locations_all_time_scales.csv"))

In [11]:
df_all

,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,actual_evapo_mmmonth,...,avg_rh_perc,dailyGDD_diff_cday,diurnal_temp_range_c,precip/dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,final_season,freezing_dd_cday,heat_dd_cday,location
0,2001-10-01,274,0,Emergence,0.0,25.25,7.35,1.41,4.0,63.5,...,42.25,0.00,17.9,0.0,0.00,0,Preseason,0.00,2.46,farmington
1,2001-10-02,275,1,Emergence,0.0,21.65,8.05,1.18,3.2,NaN,...,45.15,1.45,13.6,0.0,0.00,0,Preseason,0.00,0.00,farmington
2,2001-10-03,276,2,Emergence,0.0,19.75,5.65,1.11,3.0,NaN,...,38.00,2.15,14.1,0.0,0.00,0,Preseason,0.00,0.00,farmington
3,2001-10-04,277,3,Emergence,0.0,17.25,2.05,0.93,3.7,NaN,...,39.15,3.05,15.2,0.0,0.00,0,Preseason,2.18,0.00,farmington
4,2001-10-05,278,4,Emergence,0.0,17.45,-0.65,0.88,2.4,NaN,...,44.20,1.25,18.1,0.0,0.00,0,Preseason,4.88,0.00,farmington
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128519,2023-08-05,217,216,Beyond Maturity,0.0,27.15,14.75,1.23,4.7,NaN,...,59.45,1.95,12.4,0.0,1900.50,0,Beyond Maturity,0.00,0.00,kent
128520,2023-08-06,218,217,Beyond Maturity,0.0,25.75,14.05,0.96,4.8,NaN,...,67.20,1.05,11.7,0.0,1920.40,0,Beyond Maturity,0.40,0.00,kent
128521,2023-08-07,219,218,Beyond Maturity,0.0,28.15,12.55,1.15,5.5,NaN,...,67.85,0.45,15.6,0.0,1940.75,0,Beyond Maturity,1.90,0.00,kent
128522,2023-08-08,220,219,Beyond Maturity,0.0,31.35,12.65,1.84,5.9,NaN,...,51.55,1.65,18.7,0.0,1962.75,0,Beyond Maturity,1.80,1.35,kent


In [13]:
dates = pd.read_excel(data_base +"planting_harvest_dates.xlsx")
dates.head()

,Year,Location,Planting date,Harvest date,Latitude,Longitude
0,2002,Almira,2002-04-08,2002-08-12,47.840633,-118.894615
1,2003,Almira,2003-04-29,2003-08-20,47.840633,-118.894615
2,2004,Almira,2004-04-01,2004-08-12,47.840633,-118.894615
3,2005,Almira,2005-04-14,2005-08-16,47.874718,-118.836548
4,2006,Almira,2006-04-18,2006-08-10,47.816368,-118.865517


In [23]:
# Convert dates
df_all["date"] = pd.to_datetime(df_all["date"])

dates["Planting date"] = pd.to_datetime(dates["Planting date"])
dates["Harvest date"] = pd.to_datetime(dates["Harvest date"])
dates["Year"] = dates["Year"].astype(int)

In [50]:
def normalize_location(s):
    return (
        s.str.lower()
         .str.strip()
         .str.replace("_", " ", regex=False)
        .str.replace(".", "", regex=False)
    )

df_all["location"] = normalize_location(df_all["location"])
dates["Location"] = normalize_location(dates["Location"])

In [51]:
df_all["location"]

0         farmington
1         farmington
2         farmington
3         farmington
4         farmington
             ...    
128519          kent
128520          kent
128521          kent
128522          kent
128523          kent
Name: location, Length: 128524, dtype: object

In [52]:
dates["Location"]

0         almira
1         almira
2         almira
3         almira
4         almira
         ...    
512    mt vernon
513    mt vernon
514    mt vernon
515    mt vernon
516    mt vernon
Name: Location, Length: 517, dtype: object

In [48]:
exclude_cols = {
    "date", "doy", "dap", "stage", "final_season", "location"
}

climate_vars = [
    c for c in df_all.columns
    if c not in exclude_cols and pd.api.types.is_numeric_dtype(df_all[c])
]


In [53]:
from datetime import datetime

filtered_daily = []

for _, row in dates.iterrows():

    location = row["Location"]
    year = row["Year"]
    harvest_date = row["Harvest date"]

    start_date = datetime(year - 1, 9, 1)

    mask = (
        (df_all["location"] == location) &
        (df_all["date"] >= start_date) &
        (df_all["date"] < harvest_date)
    )

    window_df = df_all.loc[mask].copy()

    if window_df.empty:
        print(f"⚠️ No data for {location}, {year}")
        continue

    window_df["season_year"] = year
    window_df["season_start"] = start_date
    window_df["harvest_date"] = harvest_date

    filtered_daily.append(window_df)


⚠️ No data for moses lake, 2004
⚠️ No data for moses lake, 2005
⚠️ No data for moses lake, 2006
⚠️ No data for moses lake, 2007
⚠️ No data for moses lake, 2009
⚠️ No data for moses lake, 2010
⚠️ No data for moses lake, 2011
⚠️ No data for moses lake, 2012
⚠️ No data for moses lake, 2013
⚠️ No data for moses lake, 2014
⚠️ No data for moses lake, 2016
⚠️ No data for moses lake, 2017
⚠️ No data for moses lake, 2018
⚠️ No data for moses lake, 2019
⚠️ No data for moses lake, 2020
⚠️ No data for moses lake, 2021
⚠️ No data for moses lake, 2022
⚠️ No data for moses lake, 2023
⚠️ No data for pasco, 2016
⚠️ No data for pasco, 2017
⚠️ No data for pasco, 2018
⚠️ No data for pasco, 2019
⚠️ No data for pasco, 2020
⚠️ No data for pasco, 2021
⚠️ No data for pasco, 2023
⚠️ No data for royal slope, 2002
⚠️ No data for athena, 2023
⚠️ No data for klamath falls, 2020
⚠️ No data for klamath falls, 2023
⚠️ No data for la grande, 2022
⚠️ No data for la grande, 2023
⚠️ No data for madras, 2021
⚠️ No data for

In [54]:
filtered_daily_df = (
    pd.concat(filtered_daily, ignore_index=True)
    .sort_values(["location", "date"])
)


In [57]:
filtered_daily_df.to_csv(
    os.path.join(data_base, "filtered_daily_september_to_harvest.csv"),
    index=False)

# Convert daily raw from october to harvest to padded time series data

In [61]:
df = filtered_daily_df.copy()
df["date"] = pd.to_datetime(df["date"])
df.head(2)

,date,doy,dap,stage,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,actual_evapo_mmmonth,...,precip/dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,final_season,freezing_dd_cday,heat_dd_cday,location,season_year,season_start,harvest_date
0,2001-10-01,274,0,Emergence,0.0,24.15,8.65,1.37,3.3,25.7,...,0.0,0.0,0,Preseason,0.0,1.36,almira,2002,2001-09-01,2002-08-12
1,2001-10-02,275,1,Emergence,0.0,20.25,7.75,1.12,3.2,NaN,...,0.0,0.0,0,Preseason,0.0,0.00,almira,2002,2001-09-01,2002-08-12


In [139]:
exclude_cols = {
    "date", "location", "season_year","doy","dap",
    "season", "stage", "season_start", "harvest_date",
    "actual_evapo_mmmonth", "actual_evapo_mmmonth", "actual_evapo_mmmonth",  "actual_evapo_mmmonth"
}

climate_vars = [
    c for c in df.columns
    if c not in exclude_cols and pd.api.types.is_numeric_dtype(df[c])
]

In [140]:
climate_vars

['precip_mmday',
 'max_temp_c',
 'min_temp_c',
 'vpd_kpa',
 'potential_evapo_mmday',
 'palmer_drought_unitless',
 'std_precip_evap_unitless',
 'soil_moisture_mm',
 'wind_speed_ms',
 'sr_wm2',
 'max_rh_perc',
 'min_rh_perc',
 'specific_humidity_kgkg',
 'avg_temp_c',
 'avg_rh_perc',
 'dailyGDD_diff_cday',
 'diurnal_temp_range_c',
 'precip/dtr_mmdayc',
 'cum_gdd_cday',
 'wet_day_frequency_days',
 'freezing_dd_cday',
 'heat_dd_cday']

In [141]:
df_ts = (
    df
    .groupby(["location", "season_year"])
    .apply(
        lambda x: {
            "date": x["date"].tolist(),
            **{v: x[v].tolist() for v in climate_vars}
        }
    )
    .reset_index(name="TS")
)


In [144]:
df_ts.loc[0, "TS"]["std_precip_evap_unitless"][:5]


[nan, -0.2400000000000002, nan, nan, nan]

# df_ts to dataframe

In [145]:

sample_ts = df_ts.loc[0, "TS"]

climate_vars = [
    k for k in sample_ts.keys()
    if k != "date"
]

In [146]:
rows = []

for _, row in df_ts.iterrows():
    loc = row["location"]
    yr = row["season_year"]
    ts = row["TS"]

    date_series = pd.to_datetime(ts["date"])

    out = {
        "location": loc,
        "season_year": yr,
        "date": pd.Series(date_series)   # ONLY place date lives
    }

    for v in ts.keys():
        if v == "date":
            continue
        out[v] = pd.Series(ts[v])        # values only, NO index

    rows.append(out)

df_timeseries = pd.DataFrame(rows)


In [147]:
df_timeseries = pd.DataFrame(rows)


In [150]:
df_timeseries


,location,season_year,date,precip_mmday,max_temp_c,min_temp_c,vpd_kpa,potential_evapo_mmday,palmer_drought_unitless,std_precip_evap_unitless,...,specific_humidity_kgkg,avg_temp_c,avg_rh_perc,dailyGDD_diff_cday,diurnal_temp_range_c,precip/dtr_mmdayc,cum_gdd_cday,wet_day_frequency_days,freezing_dd_cday,heat_dd_cday
0,almira,2002,0 2001-10-01 1 2001-10-02 2 2001-1...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 24.15 1 20.25 2 18.45 3 ...,0 8.65 1 7.75 2 6.35 3 ...,0 1.37 1 1.12 2 1.08 3 0.9...,0 3.3 1 3.2 2 3.2 3 3.6 4 ...,0 NaN 1 -3.58 2 NaN 3 Na...,0 NaN 1 -0.24 2 NaN 3 Na...,...,0 0.00466 1 0.00401 2 0.00314 3...,0 16.40 1 14.00 2 12.40 3 ...,0 42.95 1 43.55 2 38.70 3 ...,0 0.00 1 2.40 2 1.60 3 2.9...,0 15.5 1 12.5 2 12.1 3 13....,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 0.00 1 0.00 2 0.00 3...,0 0 1 0 2 0 3 0 4 0 ...,0 0.00 1 0.00 2 0.00 3 1.2...,0 1.36 1 0.00 2 0.00 3 0.0...
1,almira,2003,0 2002-10-01 1 2002-10-02 2 2002-1...,0 0.0 1 0.0 2 1.8 3 0.3 4 ...,0 13.95 1 14.75 2 12.95 3 ...,0 1.65 1 -0.65 2 6.45 3 ...,0 0.55 1 0.55 2 0.38 3 0.3...,0 2.3 1 2.1 2 1.3 3 1.8 4 ...,0 NaN 1 -2.42 2 NaN 3 Na...,0 NaN 1 -0.10 2 NaN 3 Na...,...,0 0.00398 1 0.00389 2 0.00564 3...,0 7.80 1 7.05 2 9.70 3 ...,0 60.30 1 63.50 2 53.00 3 ...,0 12.85 1 0.75 2 2.65 3 ...,0 12.3 1 15.4 2 6.5 3 15....,0 0.000000 1 0.000000 2 0.27692...,0 1949.70 1 1949.70 2 1949.70 3...,0 0 1 0 2 1 3 1 4 0 ...,0 12.8 1 15.1 2 8.0 3 12....,0 0.00 1 0.00 2 0.00 3 0.0...
2,almira,2004,0 2003-10-01 1 2003-10-02 2 2003-1...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 26.15 1 24.85 2 25.25 3 ...,0 9.65 1 11.25 2 9.45 3 ...,0 1.65 1 1.61 2 1.64 3 1.8...,0 3.8 1 4.3 2 3.4 3 2.9 4 ...,0 NaN 1 -1.26 2 NaN 3 Na...,0 NaN 1 -2.09 2 NaN 3 Na...,...,0 0.00432 1 0.00417 2 0.00375 3...,0 17.90 1 18.05 2 17.35 3 ...,0 38.45 1 35.05 2 33.85 3 ...,0 0.90 1 0.15 2 0.70 3 0.6...,0 16.5 1 13.6 2 15.8 3 16....,0 0.000000 1 0.000000 2 0.00000...,0 2108.05 1 2108.05 2 2108.05 3...,0 0 1 0 2 0 3 0 4 0 ...,0 4.8 1 3.2 2 5.0 3 4.6 4 ...,0 0.00 1 0.00 2 0.00 3 0.0...
3,almira,2005,0 2004-10-01 1 2004-10-02 2 2004-1...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 18.95 1 20.85 2 21.35 3 ...,0 3.85 1 5.95 2 7.35 3 ...,0 0.87 1 1.00 2 1.07 3 1.0...,0 2.9 1 2.2 2 2.1 3 2.7 4 ...,0 -2.10 1 NaN 2 NaN 3 Na...,0 0.17 1 NaN 2 NaN 3 Na...,...,0 0.00419 1 0.00468 2 0.00479 3...,0 11.40 1 13.40 2 14.35 3 ...,0 44.55 1 55.20 2 49.20 3 ...,0 15.00 1 2.00 2 0.95 3 ...,0 15.1 1 14.9 2 14.0 3 15....,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 2243.00 1 2243.00 2 2243.00 3...,0 0 1 0 2 0 3 0 4 0 ...,0 10.6 1 8.5 2 7.1 3 7....,0 0.00 1 0.00 2 0.00 3 0.0...
4,almira,2006,0 2005-10-01 1 2005-10-02 2 2005-1...,0 3.2 1 0.0 2 6.3 3 0.0 4 ...,0 12.85 1 12.05 2 11.95 3 ...,0 5.75 1 2.85 2 4.95 3 ...,0 0.28 1 0.34 2 0.30 3 0.3...,0 1.5 1 1.6 2 1.4 3 1.3 4 ...,0 NaN 1 -3.37 2 NaN 3 Na...,0 NaN 1 0.63 2 NaN 3 Na...,...,0 0.00613 1 0.00494 2 0.00554 3...,0 9.30 1 7.45 2 8.45 3 ...,0 70.30 1 74.05 2 73.75 3 ...,0 14.50 1 1.85 2 1.00 3 ...,0 7.1 1 9.2 2 7.0 3 7....,0 0.450704 1 0.000000 2 0.90000...,0 2149.65 1 2149.65 2 2149.65 3...,0 1 1 0 2 1 3 0 4 0 ...,0 8.7 1 11.6 2 9.5 3 9....,0 0.00 1 0.00 2 0.00 3 0.0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,walla walla,2020,0 2019-10-01 1 2019-10-02 2 2019-1...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 12.85 1 16.85 2 15.35 3 ...,0 1.95 1 2.05 2 6.75 3 ...,0 0.56 1 0.82 2 0.74 3 0.8...,0 2.0 1 2.8 2 2.7 3 3.7 4 ...,0 NaN 1 1.0 2 NaN 3 NaN 4 ...,0 NaN 1 0.39 2 NaN 3 Na...,...,0 0.00344 1 0.00321 2 0.00399 3...,0 7.40 1 9.45 2 11.05 3 ...,0 53.45 1 45.45 2 47.20 3 ...,0 16.55 1 2.05 2 1.60 3 ...,0 10.9 1 14.8 2 8.6 3 7....,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 2448.40 1 2448.40 2 2448.40 3...,0 0 1 0 2 0 3 0 4 0 ...,0 12.5 1 12.4 2 7.7 3 5....,0 0.00 1 0.00 2 0.00 3 0.0...
394,walla walla,2021,0 2020-10-01 1 2020-10-02 2 2020-1...,0 0.0 1 0.0 2 0.0 3 0.0 4 ...,0 25.85 1 26.05 2 26.25 3 ...,0 10.05 1 10.35 2 10.35 3 ...,0 1.40 1 1.40 2 1.39 3 1.4...,0 3.0 1 3.0 2 2.6 3 4.5 4 ...,0 -1.87 1 NaN 2 NaN 3 Na...,

In [151]:
df_timeseries.loc[0, "max_temp_c"]
# chechk type 
type(df_timeseries.loc[0, "max_temp_c"])


pandas.core.series.Series